<h1 align=center><font size = 5>The Battle Of Neighborhoods Week 1</font></h1>

# Introduction

The United Arab Emirates (UAE; Arabic: دولة الإمارات العربية المتحدة‎ Dawlat al-ʾImārāt al-ʿArabīyyah al-Muttaḥidah), sometimes simply called the Emirates (Arabic: الإمارات‎ al-ʾImārāt), is the most modern and advance countries across the Middle East region often referred to as the Business Hub of Middle East. The UAE is a federation of seven emirates consisting of Abu Dhabi, Ajman, Dubai, Fujairah, Ras al-Khaimah, Sharjah and Umm al-Quwain. 

Dubai (/duːˈbaɪ/ doo-BY; Arabic: دبي‎ Dubay, Gulf Arabic: Arabic pronunciation: [dʊˈbɑj]) is the largest and most populous city in the United Arab Emirates (hereinafter referred to as UAE). The UAE is the most modern and advanced countries across the Middle East region and is also the central business hub location for local, regional and global businesses. Dubai is one of the 7 states in the UAE and welcomes over 30000 passengers each day through its state of the art world class airports and hosts over 60% of the entire expatriate population across the country. 

### Objective

Being a very busy city, one of the most challenging aspects is to find the best neighborhood in Dubai. A neighborhood that is in close proximity of the work location, within the budget, offers quality schooling and other required facilities. 

Our project is envisaged to analyze and classify areas of Dubai by leveraging Foursquare APIs and Machine Learning techniques such as Segmentation and Clustering. The final objective of this project is to Segment/Classify areas of Dubai based on Common Places captured from Foursquare APIs.

- Similarities and Dissimilarities between the Sectors of Dubai
- Classification of Areas by proximity, Residential, Tourism and Other